# Read video

In [1]:
%matplotlib inline
from PIL import Image
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

im = Image.open("catgif.gif")
width, height = im.size
print(f"Image size: {width} x {height} pixels")

file_format = im.format
print(f"File format: {file_format}")

color_mode = im.mode
print(f"Color mode: {color_mode}")

"""
# Convertir a modo "RGB"
im_rgb = im.convert("RGB")

# Guardar la nueva imagen en disco
im_rgb.save("ejemplo_rgb.gif")
"""
def run_image(im):
    try: 
        while 1:
            im.seek(im.tell()+1)
            print(im.tell())
            a = np.array(im)
            print(a.shape) # do something to im
    except EOFError:
        pass # end of sequence
    return a

a=run_image(im)



Image size: 300 x 382 pixels
File format: GIF
Color mode: P
1
(382, 300, 4)
2
(382, 300, 4)
3
(382, 300, 4)
4
(382, 300, 4)
5
(382, 300, 4)
6
(382, 300, 4)
7
(382, 300, 4)
8
(382, 300, 4)
9
(382, 300, 4)
10
(382, 300, 4)
11
(382, 300, 4)
12
(382, 300, 4)
13
(382, 300, 4)
14
(382, 300, 4)
15
(382, 300, 4)
16
(382, 300, 4)
17
(382, 300, 4)
18
(382, 300, 4)
19
(382, 300, 4)
20
(382, 300, 4)
21
(382, 300, 4)
22
(382, 300, 4)
23
(382, 300, 4)
24
(382, 300, 4)
25
(382, 300, 4)
26
(382, 300, 4)
27
(382, 300, 4)
28
(382, 300, 4)
29
(382, 300, 4)
30
(382, 300, 4)
31
(382, 300, 4)
32
(382, 300, 4)
33
(382, 300, 4)
34
(382, 300, 4)
35
(382, 300, 4)
36
(382, 300, 4)
37
(382, 300, 4)
38
(382, 300, 4)
39
(382, 300, 4)
40
(382, 300, 4)
41
(382, 300, 4)
42
(382, 300, 4)
43
(382, 300, 4)
44
(382, 300, 4)
45
(382, 300, 4)
46
(382, 300, 4)
47
(382, 300, 4)
48
(382, 300, 4)
49
(382, 300, 4)
50
(382, 300, 4)


1. Filter

In [2]:
# Convertir la imagen a escala de grises
im_grayscale = im.convert("L")
print(im_grayscale)

im = Image.open("catgif.gif")
try:
    while True:
        # Leer la siguiente imagen del GIF
        im.seek(im.tell() + 1)
        
        # Convertir la imagen a escala de grises
        im_grayscale = im.convert("L")
        # Convertir la imagen a un tensor de NumPy
        tensor = np.array(im_grayscale)

        # Imprimir la forma del tensor resultante
        print(tensor.shape)
        # Realizar operaciones con el tensor

except EOFError:
    pass





<PIL.Image.Image image mode=L size=300x382 at 0x1A9B16A2A90>
(382, 300)
(382, 300)
(382, 300)
(382, 300)
(382, 300)
(382, 300)
(382, 300)
(382, 300)
(382, 300)
(382, 300)
(382, 300)
(382, 300)
(382, 300)
(382, 300)
(382, 300)
(382, 300)
(382, 300)
(382, 300)
(382, 300)
(382, 300)
(382, 300)
(382, 300)
(382, 300)
(382, 300)
(382, 300)
(382, 300)
(382, 300)
(382, 300)
(382, 300)
(382, 300)
(382, 300)
(382, 300)
(382, 300)
(382, 300)
(382, 300)
(382, 300)
(382, 300)
(382, 300)
(382, 300)
(382, 300)
(382, 300)
(382, 300)
(382, 300)
(382, 300)
(382, 300)
(382, 300)
(382, 300)
(382, 300)
(382, 300)
(382, 300)


In [15]:
from PIL import Image
import numpy as np
import imageio

# Abrir el archivo GIF
im = Image.open("catgif.gif")

def filter(im, scale):
    fotogramas_modificados=[]
    try:
        while True:
            # Leer la siguiente imagen del GIF
            im.seek(im.tell() + 1)

            # Obtener la paleta de colores del GIF
            palette = im.getpalette()

            # Convertir la imagen a RGB utilizando la paleta de colores
            im_rgb = im.convert("RGB").quantize(palette=palette)

            # Convertir la imagen a escala de grises
            im_grayscale = im_rgb.convert("L")

            # Convertir la imagen a un tensor de NumPy
            tensor = np.array(im_grayscale)

            # Filtro de oscurecimiento e iluminación
            scale_factor = scale  # factor de escala (entre 0 y 1 para iluminar, mayor que 1 para oscurecer)
            tensor_filtered = tensor * scale_factor

            im_filtered = Image.fromarray(tensor_filtered)
            # Convertir la imagen al modo RGB
            im_filtered = im_filtered.convert("RGB")
            # Guardar la imagen filtrada
            im_filtered.save(f"frame_{im.tell()}.png")
            
            # Añadir el fotograma modificado a la lista
            fotogramas_modificados.append(im_filtered)

    except EOFError:
        return fotogramas_modificados
        pass


fotogramas_modificados_dark=filter(im, 0.1)
im = Image.open("catgif.gif")
fotogramas_modificados_ligh=filter(im, 0.99999999)
# Guardar los fotogramas modificados en un nuevo archivo GIF
imageio.mimsave("catgif_oscurecido.gif", fotogramas_modificados_dark)
imageio.mimsave("catgif_claro.gif", fotogramas_modificados_ligh)

